Question 3 -

Train a Pure CNN with less than 10000 trainable parameters using the MNIST
Dataset having minimum validation accuracy of 99.40%
Note -
1. Code comments should be given for proper code understanding.
2. Implement in both PyTorch and Tensorflow respectively

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs = 10
batch_size = 128
learning_rate = 0.001

# MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, transform=transform, download=True
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=False
)

# Define the CNN model
class PureCNN(nn.Module):
    def __init__(self):
        super(PureCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        out = nn.functional.relu(self.conv1(x))
        out = nn.functional.max_pool2d(out, 2)
        out = nn.functional.relu(self.conv2(out))
        out = nn.functional.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = nn.functional.relu(self.fc1(out))
        out = self.fc2(out)
        return out

# Instantiate the model
model = PureCNN().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}")

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

100%|██████████| 9912422/9912422 [00:00<00:00, 90979677.89it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 21129547.15it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26267799.78it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 21285507.00it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/10], Step [100/469], Loss: 0.3635
Epoch [1/10], Step [200/469], Loss: 0.1496
Epoch [1/10], Step [300/469], Loss: 0.1418
Epoch [1/10], Step [400/469], Loss: 0.0718
Epoch [2/10], Step [100/469], Loss: 0.0756
Epoch [2/10], Step [200/469], Loss: 0.0291
Epoch [2/10], Step [300/469], Loss: 0.0778
Epoch [2/10], Step [400/469], Loss: 0.0915
Epoch [3/10], Step [100/469], Loss: 0.0221
Epoch [3/10], Step [200/469], Loss: 0.0482
Epoch [3/10], Step [300/469], Loss: 0.0562
Epoch [3/10], Step [400/469], Loss: 0.0436
Epoch [4/10], Step [100/469], Loss: 0.0197
Epoch [4/10], Step [200/469], Loss: 0.0200
Epoch [4/10], Step [300/469], Loss: 0.0188
Epoch [4/10], Step [400/469], Loss: 0.0216
Epoch [5/10], Step [100/469], Loss: 0.0072
Epoch [5/10], Step [200/469], Loss: 0.0835
Epoch [5/10], Step [300/469], Loss: 0.0224
Epoch [5/10], Step [400/469], Loss: 0.0899
Epoch [6/10], Step [100/469], Loss: 0.0054
Epoch [6/10], Step [20

Answer-b

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# Hyperparameters
num_epochs = 10
batch_size = 128
learning_rate = 0.001

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Define the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(16, 3, activation="relu", input_shape=(28, 28, 1)),
    layers.MaxPooling2D(2),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10),
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs, verbose=1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/10
469/469 [==============================] - 5s 4ms/step - loss: 0.2891 - accuracy: 0.9180
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0671 - accuracy: 0.9796
Epoch 3/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0462 - accuracy: 0.9856
Epoch 4/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0364 - accuracy: 0.9888
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0284 - accuracy: 0.9911
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0246 - accuracy: 0.9924
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0195 - accuracy: 0.9939
Epoch 8/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0168 - accuracy: 0.9946
Epoch 9/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0144 - accuracy: 0.9954
Epoch 10/10
469/469 [==============